In [ ]:
import os 

import numpy as np
import scipy.stats as sps
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random

import sys, os
sys.path += [os.path.abspath(os.pardir + '/code')]
print(sys.path)

from experiment import init_random_state, BanditLoopExperiment, get_ts_model

sns.set(font_scale=1.2, palette='Set2') 

In [ ]:
def draw_posteriori(grid, distr_class, post_params, obj, steps, xlim=None):
    '''Рисует серию графиков апостериорных плотностей.
    
    :param grid: сетка для построения графика
    :param distr_class: класс распределений из scipy.stats
    :param post_params: параметры апостериорных распределений 
        shape=(размер выборки, кол-во параметров)
    '''
    
    sns.set(font_scale=1.3)
    
    size = post_params.shape[0] - 1
    plt.figure(figsize=(12, 7))
    for n, t in enumerate(steps):
        plt.plot(grid, 
                 distr_class(post_params[n]).pdf(grid) \
                     if np.isscalar(post_params[n]) \
                     else distr_class(*post_params[n]).pdf(grid), 
                 label='t={}: {}'.format(t, np.round(post_params[n], 3)), 
                 lw=2.5,
                 color=(1-n/size, n/size, 0))
    plt.title(f'Апостериорное распределение для объекта {obj} в зависимости от шага')
    plt.grid(ls=':')
    plt.legend(fontsize=12)
    plt.xlim(xlim)
    plt.show()

In [ ]:
seed = 42

p = 0.8
Q = 1
ws = np.linspace(40, 50, 11)

T = 1000
M = 5
l = 2

In [ ]:
interests, TS_paramss, responses = [], [], []

In [ ]:
for w in ws:
    init_random_state(seed)
    
    bandit = lambda: get_ts_model(M=M, l=l)
    exp = BanditLoopExperiment(bandit, "TS bandit")
    exp.prepare(w=w, Q=Q, p=p)
    exp.run_experiment(T=T)
    
    results = exp.get_as_np()
    interests.append(results.interest)
    TS_paramss.append(results.TS_params)
    responses.append(results.response)

In [ ]:
sum_responces = []
for  i, w in enumerate(ws):
    sum_responces.append(np.cumsum(responses[i].sum(axis=1)))
plt.figure(figsize=(12, 6))

for i, w in enumerate(ws):
    plt.plot(np.arange(1, T+1), sum_responces[i], label=f'Рекомендательная система с шумом w = {w}')
plt.plot(np.arange(T)*l, label='идеальный случай', linestyle='--')
plt.title('Зависимость суммы откликов от времени')
plt.ylabel('Сумма откликов')
plt.xlabel('Шаг')
plt.legend()

In [ ]:
plt.figure(figsize=(18, 18))
for m in range(M):
    plt.subplot(M // 2 + 1, 2, m+1)
    for i, w in enumerate(ws):
        plt.plot(interests[i][:, m], label=f'с шумом w = {w}')
    plt.title(f'интерес к {m} объекту')
    plt.ylabel('интерес')
    plt.xlabel('Шаг')
    plt.legend()

    plt.tight_layout()

In [ ]:
plt.figure(figsize=(16, 8))
for i, w in enumerate(ws):
    plt.plot(np.linalg.norm(interests[i] - interests[i][0], axis=1)**2, label=f'с шумом w = {w}')
plt.ylabel(r'$\|\mu_t - \mu_0 \|^2$')
plt.legend()
plt.xlabel('Шаг')

In [ ]:
for i in range(M):
    draw_posteriori(np.linspace(0, 1., 100), sps.beta, TS_paramss[0][::100][:, i], obj=i, steps = range(0, T, 100))